# Aggregations and Grouping

## Prerrequisites

Install Spark and Java in VM

In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark 3.5.0
!wget -q https://apache.osuosl.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [ ]:
ls -l # check the .tgz is there

total 391020
drwxr-xr-x 1 root root      4096 Jan 11 17:02 sample_data/
-rw-r--r-- 1 root root 400395283 Sep  9 02:10 spark-3.5.0-bin-hadoop3.tgz


In [ ]:
# unzip it
!tar xf spark-3.5.0-bin-hadoop3.tgz

In [ ]:
!pip install -q findspark

In [ ]:
!pip install py4j

# For maps
!pip install folium
!pip install plotly

Define the environment

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session

---

In [ ]:
import findspark
findspark.init("spark-3.5.0-bin-hadoop3")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("Aggregations and Grouping") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.5.0'

In [ ]:
spark

In [ ]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [ ]:
!mkdir -p dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/movies.json -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/vehicles.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/characters.csv -P /dataset
!ls /dataset

characters.csv	movies.json  vehicles.csv


Read JSON file

---

In [ ]:
moviesDF = spark.read \
    .option("inferSchema", True) \
    .json("/dataset/movies.json")

In [ ]:
moviesDF.show(2, False)
print(moviesDF.schema.fields)
moviesDF.columns

+-------------+--------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+------+----------------------+------------+--------+---------------+
|Creative_Type|Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|Source|Title                 |US_DVD_Sales|US_Gross|Worldwide_Gross|
+-------------+--------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+------+----------------------+------------+--------+---------------+
|NULL         |NULL    |Gramercy   |6.1        |1071      |R          |NULL       |8000000          |12-Jun-98   |NULL                  |NULL            |NULL  |The Land Girls        |NULL        |146083  |146083         |
|NULL         |NULL    |Strand     |6.9        |207       |R          |Drama      |300000           |7-Aug-9

['Creative_Type',
 'Director',
 'Distributor',
 'IMDB_Rating',
 'IMDB_Votes',
 'MPAA_Rating',
 'Major_Genre',
 'Production_Budget',
 'Release_Date',
 'Rotten_Tomatoes_Rating',
 'Running_Time_min',
 'Source',
 'Title',
 'US_DVD_Sales',
 'US_Gross',
 'Worldwide_Gross']

## Examples

Count

In [ ]:
# df rows counting, including NULLS
moviesDF.count()

3201

In [ ]:
# using sql functions, NOT including NULLS
genresCountDF = moviesDF.select(count(col("Major_Genre")))
genresCountDF.show()

+------------------+
|count(Major_Genre)|
+------------------+
|              2926|
+------------------+



In [ ]:
directorsCountDF = moviesDF.select(count(moviesDF.Director))
directorsCountDF.show()

+---------------+
|count(Director)|
+---------------+
|           1870|
+---------------+



In [ ]:
moviesDF.select(count(moviesDF.Major_Genre).alias("countMajor"), count(moviesDF.Director)).show()

+----------+---------------+
|countMajor|count(Director)|
+----------+---------------+
|      2926|           1870|
+----------+---------------+



In [ ]:
#using SQL syntax
moviesDF.select(expr("count(Director)")).show()
moviesDF.selectExpr("count(Director) as count").show()

+---------------+
|count(Director)|
+---------------+
|           1870|
+---------------+

+-----+
|count|
+-----+
| 1870|
+-----+



In [ ]:
# using SQL
moviesDF.createOrReplaceTempView("movies")

In [ ]:
spark.sql("select count(Director) from movies").show()

+---------------+
|count(Director)|
+---------------+
|           1870|
+---------------+



In [ ]:
spark.sql("select count(Director) as countDirector, count(Major_Genre) from movies").show()

+-------------+------------------+
|countDirector|count(Major_Genre)|
+-------------+------------------+
|         1870|              2926|
+-------------+------------------+



Count Distinct

In [ ]:
moviesDF.select(countDistinct(moviesDF.Major_Genre)).show()

+---------------------------+
|count(DISTINCT Major_Genre)|
+---------------------------+
|                         12|
+---------------------------+



In [ ]:
spark.sql("select count(distinct Major_Genre) from movies").show()

+---------------------------+
|count(DISTINCT Major_Genre)|
+---------------------------+
|                         12|
+---------------------------+



Min and max

In [ ]:
moviesDF.select(min(moviesDF.Production_Budget), max(moviesDF.Production_Budget)).show()

+----------------------+----------------------+
|min(Production_Budget)|max(Production_Budget)|
+----------------------+----------------------+
|                   218|             300000000|
+----------------------+----------------------+



In [ ]:
spark.sql("select min(Production_Budget) from movies").show()

+----------------------+
|min(Production_Budget)|
+----------------------+
|                   218|
+----------------------+



Sum

In [ ]:
moviesDF.select(sum(moviesDF.US_DVD_Sales).alias("salesUS")).show()
moviesDF.selectExpr("sum(US_DVD_Sales) as sales").show()

+-----------+
|    salesUS|
+-----------+
|19684472405|
+-----------+

+-----------+
|      sales|
+-----------+
|19684472405|
+-----------+



Average

In [ ]:
moviesDF.select(avg(moviesDF.Production_Budget)).show()
spark.sql("select avg(Production_Budget) from movies").show()

+----------------------+
|avg(Production_Budget)|
+----------------------+
|    3.10691714484375E7|
+----------------------+

+----------------------+
|avg(Production_Budget)|
+----------------------+
|    3.10691714484375E7|
+----------------------+



Stats

In [ ]:
moviesDF.select(mean(moviesDF.Rotten_Tomatoes_Rating)).show()
moviesDF.select(stddev(moviesDF.Rotten_Tomatoes_Rating)).show()

+---------------------------+
|avg(Rotten_Tomatoes_Rating)|
+---------------------------+
|          54.33692373976734|
+---------------------------+

+------------------------------+
|stddev(Rotten_Tomatoes_Rating)|
+------------------------------+
|             28.07659263787602|
+------------------------------+



### Grouping

---

In [ ]:
countByGenreGF = moviesDF.groupBy(moviesDF.Major_Genre).count().orderBy("count")
countByGenreGF.show()

+-------------------+-----+
|        Major_Genre|count|
+-------------------+-----+
|Concert/Performance|    5|
|       Black Comedy|   36|
|            Western|   36|
|        Documentary|   43|
|            Musical|   53|
|    Romantic Comedy|  137|
|             Horror|  219|
|  Thriller/Suspense|  239|
|          Adventure|  274|
|               NULL|  275|
|             Action|  420|
|             Comedy|  675|
|              Drama|  789|
+-------------------+-----+



In [ ]:
spark.sql("select Major_Genre, count(Major_Genre) as count from movies where Major_Genre is not null group by Major_Genre order by count").show()

+-------------------+-----+
|        Major_Genre|count|
+-------------------+-----+
|Concert/Performance|    5|
|       Black Comedy|   36|
|            Western|   36|
|        Documentary|   43|
|            Musical|   53|
|    Romantic Comedy|  137|
|             Horror|  219|
|  Thriller/Suspense|  239|
|          Adventure|  274|
|             Action|  420|
|             Comedy|  675|
|              Drama|  789|
+-------------------+-----+



In [ ]:
avgRatingByGenreDF = moviesDF.groupBy(col("Major_Genre")).avg("IMDB_Rating").orderBy(col("avg(IMDB_Rating)").desc())
avgRatingByGenreDF.show()

+-------------------+------------------+
|        Major_Genre|  avg(IMDB_Rating)|
+-------------------+------------------+
|        Documentary| 6.997297297297298|
|            Western| 6.842857142857142|
|       Black Comedy|6.8187500000000005|
|              Drama| 6.773441734417339|
|               NULL|  6.50082644628099|
|            Musical|             6.448|
|  Thriller/Suspense| 6.360944206008582|
|          Adventure| 6.345019920318729|
|Concert/Performance|             6.325|
|             Action| 6.114795918367349|
|    Romantic Comedy| 5.873076923076922|
|             Comedy| 5.853858267716529|
|             Horror|5.6760765550239185|
+-------------------+------------------+



In [ ]:
moviesDF.groupBy(col("Major_Genre")).agg(avg("IMDB_Rating") \
    .alias("avg")).orderBy(col("avg").desc()).show()

+-------------------+------------------+
|        Major_Genre|               avg|
+-------------------+------------------+
|        Documentary| 6.997297297297298|
|            Western| 6.842857142857142|
|       Black Comedy|6.8187500000000005|
|              Drama| 6.773441734417339|
|               NULL|  6.50082644628099|
|            Musical|             6.448|
|  Thriller/Suspense| 6.360944206008582|
|          Adventure| 6.345019920318729|
|Concert/Performance|             6.325|
|             Action| 6.114795918367349|
|    Romantic Comedy| 5.873076923076922|
|             Comedy| 5.853858267716529|
|             Horror|5.6760765550239185|
+-------------------+------------------+



In [ ]:
aggregationsByGenreDF = moviesDF.groupBy("Major_Genre") \
    .agg(
        count("*").alias("N_Movies"),
        avg("IMDB_Rating").alias("rating")
    ) \
    .orderBy(col("rating").desc()).show()

+-------------------+--------+------------------+
|        Major_Genre|N_Movies|            rating|
+-------------------+--------+------------------+
|        Documentary|      43| 6.997297297297298|
|            Western|      36| 6.842857142857142|
|       Black Comedy|      36|6.8187500000000005|
|              Drama|     789| 6.773441734417339|
|               NULL|     275|  6.50082644628099|
|            Musical|      53|             6.448|
|  Thriller/Suspense|     239| 6.360944206008582|
|          Adventure|     274| 6.345019920318729|
|Concert/Performance|       5|             6.325|
|             Action|     420| 6.114795918367349|
|    Romantic Comedy|     137| 5.873076923076922|
|             Comedy|     675| 5.853858267716529|
|             Horror|     219|5.6760765550239185|
+-------------------+--------+------------------+



## Exercises
   1. Sum up all the worldwide profits of ALL the movies in the DF. Then sum the worldwide profits per director
   2. Count how many distinct directors we have
   3. Show the mean and standard deviation of US gross revenue for the movies (all the movies)
   4. Compute the average IMDB rating and the average US gross revenue PER DIRECTOR
   5. Sum up ALL the profits of ALL the movies in the DF. Then sum ALL the profits per director. Can you see null values? Why? How you can solve it?


Exercise 1

In [ ]:
ls/dataset

characters.csv  movies.json  vehicles.csv


In [ ]:
from pyspark.sql.functions import sum as spark_sum

In [ ]:
moviesDF = spark.read.option("inferSchema", True).json("/dataset/movies.json")

In [ ]:
moviesDF = spark.read.option("header", "true").option("delimiter", ";").json("/dataset/movies.json")
moviesDF.show()

+--------------------+-----------------+--------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|       Creative_Type|         Director|   Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|
+--------------------+-----------------+--------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|                NULL|             NULL|      Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|                  NULL|            NULL|               NULL|      The Land Girls|        NULL|  146083|     

In [ ]:
moviesDF.select("Title", "Worldwide_Gross").show()

+--------------------+---------------+
|               Title|Worldwide_Gross|
+--------------------+---------------+
|      The Land Girls|         146083|
|First Love, Last ...|          10876|
|I Married a Stran...|         203134|
|Let's Talk About Sex|         373615|
|                Slam|        1087521|
| Mississippi Mermaid|        2624551|
|           Following|          44705|
|             Foolish|        6026908|
|             Pirates|        6341825|
|     Duel in the Sun|       20400000|
|           Tom Jones|       37600000|
|             Oliver!|       37402877|
|To Kill A Mocking...|       13129846|
|    Tora, Tora, Tora|       29548291|
|   Hollywood Shuffle|        5228617|
|Over the Hill to ...|        3000000|
|              Wilson|        2000000|
|        Darling Lili|        5000000|
|The Ten Commandments|       80000000|
|        12 Angry Men|              0|
+--------------------+---------------+
only showing top 20 rows



In [ ]:
total_worldwide_profit = moviesDF.agg(spark_sum("Worldwide_Gross").alias("Total_Worldwide_Profit")).collect()[0]["Total_Worldwide_Profit"]


In [ ]:
result_df = spark.createDataFrame([(total_worldwide_profit,)], ["Total_Worldwide_Profit"])


In [ ]:
result_df.show()

+----------------------+
|Total_Worldwide_Profit|
+----------------------+
|          272586820052|
+----------------------+



In [ ]:
director_worldwide_profit = moviesDF.groupBy("Director").agg(spark_sum("Worldwide_Gross").alias("Director_Worldwide_Profit"))

In [ ]:
director_worldwide_profit.show(truncate=False)

+-------------------+-------------------------+
|Director           |Director_Worldwide_Profit|
+-------------------+-------------------------+
|John Milius        |52735525                 |
|Jim Jarmusch       |6030230                  |
|Sheldon Lettich    |29090445                 |
|Chan-wook Park     |23471871                 |
|King Vidor         |73702232                 |
|Jared Hess         |145338949                |
|Michael Curtiz     |25462500                 |
|Steven Seagal      |38590458                 |
|Bradley Rust Grey  |25572                    |
|Jeff Burr          |1355728                  |
|Bob Rafelson       |45283350                 |
|Paul Michael Glaser|38122000                 |
|Griffin Dunne      |68450430                 |
|Mark Waters        |563693780                |
|Richard Fleischer  |91054152                 |
|Akira Kurosawa     |320592                   |
|Mira Nair          |79806867                 |
|Denzel Washington  |53593730           

Exercise 2

In [ ]:
director_count = moviesDF.select(countDistinct("Director").alias("DistinctDirectors")).collect()[0]["DistinctDirectors"]

In [ ]:
result_df = spark.createDataFrame([(director_count,)], ["TotalDistinctDirectors"])

In [ ]:
result_df.show(truncate=False)

+----------------------+
|TotalDistinctDirectors|
+----------------------+
|550                   |
+----------------------+



Exercise 3

In [ ]:
from pyspark.sql.functions import avg, col

In [ ]:
worldwide_gross_mean = moviesDF.select(mean(col("Worldwide_Gross")).alias("Mean_Worldwide_Gross"))

In [ ]:
worldwide_gross_mean.show(truncate=False)

+--------------------+
|Mean_Worldwide_Gross|
+--------------------+
|8.534340014151534E7 |
+--------------------+



In [ ]:
moviesDF.select(avg(moviesDF.Worldwide_Gross)).show()

+--------------------+
|avg(Worldwide_Gross)|
+--------------------+
| 8.534340014151534E7|
+--------------------+



In [ ]:
moviesDF.select(stddev(moviesDF.Worldwide_Gross)).show()

+-----------------------+
|stddev(Worldwide_Gross)|
+-----------------------+
|   1.4994734288536233E8|
+-----------------------+



Exercise 4

In [ ]:
moviesDF.select(avg(moviesDF.IMDB_Rating)).show()

+-----------------+
| avg(IMDB_Rating)|
+-----------------+
|6.283467202141896|
+-----------------+



In [ ]:
moviesDF.select(avg(moviesDF.US_Gross)).show()

+--------------------+
|       avg(US_Gross)|
+--------------------+
|4.4002085163744524E7|
+--------------------+



In [ ]:
director_stats = moviesDF.groupBy("Director").agg(avg("IMDB_Rating").alias("Average_IMDB_Rating"), avg("US_Gross").alias("Average_US_Gross"))

In [ ]:
director_stats.show(truncate=False)

+-------------------+-------------------+----------------+
|Director           |Average_IMDB_Rating|Average_US_Gross|
+-------------------+-------------------+----------------+
|John Milius        |6.05               |2.63677625E7    |
|Jim Jarmusch       |NULL               |3330230.0       |
|Sheldon Lettich    |4.7                |2.9090445E7     |
|Chan-wook Park     |7.7                |211667.0        |
|King Vidor         |7.375              |1.5675558E7     |
|Jared Hess         |6.300000000000001  |6.23694745E7    |
|Michael Curtiz     |7.5                |8487500.0       |
|Steven Seagal      |3.8                |3.8590458E7     |
|Bradley Rust Grey  |6.2                |25572.0         |
|Jeff Burr          |5.6                |1355728.0       |
|Bob Rafelson       |6.25               |6641675.0       |
|Paul Michael Glaser|6.4                |3.8122E7        |
|Griffin Dunne      |5.5                |2.34819955E7    |
|Mark Waters        |6.380000000000001  |6.52360426E7   

Compute the average IMDB rating and the average US gross revenue PER DIRECTOR

Exercise 5:Sum up ALL the profits of ALL the movies in the DF. Then sum ALL the profits per director. Can you see null values? Why? How you can solve it?

In [ ]:
directorDF = moviesDF.select(countDistinct("Director").alias("DistinctDirectorsCount"))
directorDF.show()

+----------------------+
|DistinctDirectorsCount|
+----------------------+
|                   550|
+----------------------+



In [ ]:
spark.sql("Select Title, sum(Worldwide_Gross) as sum_wg \
            from movies \
            where Worldwide_Gross IS NOT NULL and Title IS NOT NULL \
            group by Title \
            order by Title").show()

+--------------------+---------+
|               Title|   sum_wg|
+--------------------+---------+
|         10,000 B.C.|269065678|
|      102 Dalmatians| 66941559|
|         10th & Wolf|    54702|
|               11:14|        0|
|        12 Angry Men|        0|
|           12 Rounds| 18184083|
|      13 Going On 30| 96439723|
|                1408|128529299|
|          15 Minutes| 56331864|
|           16 Blocks| 65595141|
|          16 to Life|    10744|
|                1776|        0|
|                1941| 94875000|
|    2 Fast 2 Furious|236220058|
|     2 For the Money| 27848418|
|            20 Dates|   541636|
|20,000 Leagues Un...| 36200000|
|      200 Cigarettes|  6852450|
|2001: A Space Ody...| 68700000|
|                2012|766812167|
+--------------------+---------+
only showing top 20 rows



In [ ]:
spark.sql("Select Title, sum(Worldwide_Gross) as Worldwide_Gross \
            from movies \
            where Worldwide_Gross IS NOT NULL and Title IS NOT NULL \
            group by Title \
            order by Title").show()

+--------------------+---------------+
|               Title|Worldwide_Gross|
+--------------------+---------------+
|         10,000 B.C.|      269065678|
|      102 Dalmatians|       66941559|
|         10th & Wolf|          54702|
|               11:14|              0|
|        12 Angry Men|              0|
|           12 Rounds|       18184083|
|      13 Going On 30|       96439723|
|                1408|      128529299|
|          15 Minutes|       56331864|
|           16 Blocks|       65595141|
|          16 to Life|          10744|
|                1776|              0|
|                1941|       94875000|
|    2 Fast 2 Furious|      236220058|
|     2 For the Money|       27848418|
|            20 Dates|         541636|
|20,000 Leagues Un...|       36200000|
|      200 Cigarettes|        6852450|
|2001: A Space Ody...|       68700000|
|                2012|      766812167|
+--------------------+---------------+
only showing top 20 rows

